In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('./data/steam-200k.csv')

# Print total rows
total_rows = len(df)
print("Total rows:", total_rows)
df.head()

Total rows: 199999


,151603712,The Elder Scrolls V Skyrim,purchase,1,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [2]:
# Renaming columns for better readability
cols = {'151603712': 'user_id', 'The Elder Scrolls V Skyrim': 'game_name', 'purchase': 'status', '1': 'Hourplayed'}
df.rename(columns=cols, inplace=True)

df.head()

,user_id,game_name,status,Hourplayed,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [3]:
# Drop irrelevant columns and remove duplicates
df.drop(columns=['0'], inplace=True)
df.drop_duplicates(inplace=True)

#Check for missing values and duplicates
df.isnull().sum(), df.duplicated().sum()

df.head()

,user_id,game_name,status,Hourplayed
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [4]:
# Number of unique games in the dataset
game_count = df['game_name'].nunique()
print(f"Number of unique games in the dataset: {game_count}")

# Number of unique users in the dataset
user_count = df['user_id'].nunique()
print(f"Number of unique users in the dataset: {user_count}")

Number of unique games in the dataset: 5155
Number of unique users in the dataset: 12393


In [5]:
# Convert 'user_id' to string type
df['user_id'] = df['user_id'].astype(str)

# Calculate the average playtime for each user (user-based average)
average_user_playtime = df.groupby(['user_id'], as_index=False).Hourplayed.mean()
average_user_playtime['avg_hourplayed'] = average_user_playtime['Hourplayed']
average_user_playtime.drop(columns='Hourplayed', inplace=True)

# Merge the average user playtime data with the main dataset
df = df.merge(average_user_playtime, on='user_id')

# Create a new 'rating' column based on Hourplayed compared to the user's average
condition = [
    df['Hourplayed'] >= (0.8 * df['avg_hourplayed']),
    (df['Hourplayed'] >= 0.6 * df['avg_hourplayed']) & (df['Hourplayed'] < 0.8 * df['avg_hourplayed']),
    (df['Hourplayed'] >= 0.4 * df['avg_hourplayed']) & (df['Hourplayed'] < 0.6 * df['avg_hourplayed']),
    (df['Hourplayed'] >= 0.2 * df['avg_hourplayed']) & (df['Hourplayed'] < 0.4 * df['avg_hourplayed']),
    df['Hourplayed'] >= 0
]
values = [5, 4, 3, 2, 1]
df['rating'] = np.select(condition, values)

In [6]:
# Filter the dataset to only include rows where the user has 'played' the game
# df = df[df['status'] == 'play']  # Keep only rows where status is 'play'

# Remove games with fewer than 100 ratings
df = df[df.groupby('game_name').user_id.transform(len) >= 100]

# Games to remove (because this games not available inside the metadata dataset)
games_to_remove = [
    "Dead Island Epidemic",
    "marvel heroes 2015",
    "Fallout New Vegas Courier's Stash",
    "Fallout New Vegas Dead Money",
    "Fallout New Vegas Honest Hearts",
    "The Elder Scrolls V Skyrim - Dawnguard",
    "The Elder Scrolls V Skyrim - Dragonborn",
    "The Elder Scrolls V Skyrim - Hearthfire",
    "Company of Heroes (New Steam Version)",
    "Killing Floor Mod Defence Alliance 2",
    "Skyrim High Resolution Texture Pack",
    "DiRT 3",
    "Super Meat Boy",
    "Arma 2 DayZ Mod",
    "Age of Empires II HD The Forgotten",
    "Audiosurf",
    "Borderlands",
    "Borderlands DLC Claptraps New Robot Revolution",
    "Borderlands DLC Mad Moxxi's Underdome Riot",
    "Borderlands DLC The Secret Armory of General Knoxx",
    "Borderlands DLC The Zombie Island of Dr. Ned",
    "Darksiders",
    "DiRT 3 Complete Edition",
    "Hitman Sniper Challenge",
    "Medal of Honor(TM) Multiplayer",
    "Medal of Honor(TM) Single Player",
    "Medal of Honor Pre-Order",
    "Nosgoth",
    "The Lord of the Rings War in the North",
    "Titan Quest",
    "Titan Quest Immortal Throne",
    "TERA",
    "Loadout",
    "Tactical Intervention",
    "Tom Clancy's Ghost Recon Phantoms - NA",
    "Tom Clancy's Ghost Recon Phantoms - EU",
    "War Inc. Battlezone",
    "Counter-Strike Condition Zero Deleted Scenes",
    "Infinite Crisis",
    "The Walking Dead",
    "Patch testing for Chivalry",
    "Red Orchestra 2 Heroes of Stalingrad - Single Player",
    "Sid Meier's Civilization IV Beyond the Sword",
    "Sniper Elite V2",
    "H1Z1",
    "Age of Chivalry",
    "H1Z1 Test Server",
    "Aftermath",
    "Happy Wars",
    "Ragnarok Online 2",
    "Mortal Kombat Komplete Edition",
    "Call of Duty Advanced Warfare - Multiplayer",
    "Arma 3 Zeus",
    "Blacklight Retribution",
    "Dizzel",
    "Free to Play",
    "Starbound - Unstable",
    "Don't Starve",
    "The Walking Dead Season Two",
    "Call of Duty Black Ops II - Multiplayer",
    "Call of Duty Black Ops II - Zombies",
    "HAWKEN",
    "Call of Duty Modern Warfare 3 - Multiplayer",
    "Call of Duty Modern Warfare 3",
    "Insurgency Modern Infantry Combat",
    "The Ship Single Player",
    "Football Manager 2013",
    "Dungeonland",
    "Rise of Incarnates",
    "Sid Meier's Civilization V Brave New World",
    "The Mighty Quest For Epic Loot",
    "Magicka Wizard Wars",
    "Ace of Spades",
    "Serious Sam HD The Second Encounter",
    "War of the Roses",
    "AirMech",
    "Firefall",
    "Warhammer 40,000 Dawn of War II  Retribution",
    "Warhammer 40,000 Dawn of War II - Chaos Rising",
    "Magicka Vietnam",
    "Magicka",
    "Deus Ex Human Revolution - Director's Cut",
    "Call of Duty Black Ops - Multiplayer",
    "Rising Storm/Red Orchestra 2 Multiplayer",
    "Call of Duty Ghosts - Multiplayer",
    "Rome Total War",
    "Football Manager 2012",
    "Football Manager 2015",
    "Sid Meier's Civilization V Scrambled Continents Map Pack",
    "Football Manager 2014",
    "Train Simulator",
    "Call of Duty Modern Warfare 2 - Multiplayer",
    "Arma 2 Operation Arrowhead Beta (Obsolete)",
    "Metro 2033",
    "Far Cry 3 Blood Dragon",
    "Crysis 2 Maximum Edition",
    "Infestation Survivor Stories"
]

In [7]:
# Normalize game names for filtering
def normalize_name(name):
    name = str(name).strip().lower()
    return name

games_to_remove_normalized = [normalize_name(game) for game in games_to_remove]
df['normalized_game_name'] = df['game_name'].apply(normalize_name)

# Filter out the games
df = df[~df['normalized_game_name'].isin(games_to_remove_normalized)]

# Drop the normalized column as it's no longer needed
df.drop(columns=['normalized_game_name'], inplace=True)

In [8]:
# Print final game and user count
game_count = df['game_name'].nunique()
print(f"Number of unique games in the dataset: {game_count}")
user_count = df['user_id'].nunique()
print(f"Number of unique users in the dataset: {user_count}")

Number of unique games in the dataset: 317
Number of unique users in the dataset: 11210


In [9]:
# Save cleaned data
df.to_csv('./data/Cleaned_Dataset.csv')